In [1]:
import utils.helper_functions as hf

import os

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from collections import Counter

import hyperopt
from hyperopt import hp, fmin, tpe

db_path = './db/ohlcv_ntickers_1254_2000-08-01_to_2023-12-23.pkl'
transformed_data_path = './db/classifier_transformed_data.pkl'
start_date = '2013-01-01'
test_size = 100000

# param_grid = {
#     'buying_times': ['Open'],
#     'selling_times': ['High'],
#     'target_future_days': [i for i in list(np.arange(1, 20, 3))],
#     'sizes_layer_1': [64, 128, 256],
#     'sizes_layer_2': [64, 128, 256],
#     'sizes_layer_3': [64, 128, 256],
#     'dropout_rates': [i for i in list(np.arange(0, 0.3, 0.1))],
#     'balance_data': [True, False],
#     'batch_sizes': [32, 64, 128],
#     'n_first_classes': [[0,0], [0,1], [0,2], [1,1], [1,2], [2, 2]],
#     'cumulated_probs_targets': [i for i in list(np.arange(0.4, 1, 0.2))]
# }

param_grid = {
    'buying_times': ['Open'], 'selling_times': ['High'],
    'target_future_days': [10],
    'sizes_layer_1': [256], 'sizes_layer_2': [64], 'sizes_layer_3': [64],
    'dropout_rates': [0.079], 'balance_data': [True], 'batch_sizes': [64],
    'n_first_classes': [[1,1]],
    'cumulated_probs_targets': [0.65]
}

thresholds = [1.08, 1.04, 1.02, 1]
loss_limits = [0, 0.75, 0.8, 0.85, 0.88, 0.9, 0.92, 0.94, 0.96, 0.98, 0.99] 
fee = 0.002

epochs = 2

KeyboardInterrupt: 

In [ ]:
df = pd.read_pickle(db_path)
df = hf.get_rows_after_date(df, start_date)

def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'High')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}


In [ ]:
def calculate_var(df, past_days, future_days):
    var = hf.calculate_variations(df, past_days, future_days)
    var_stacked = hf.stack(var, f'input_var_past_{past_days}d_future_{future_days}d')

    return var_stacked

def calculate_var_vs_past_ohlcv(df, df_past, past_days, title):
    var = df / df_past.shift(past_days)
    var_stacked = hf.stack(var, f'input_var_past_{title}_{past_days}d')

    return var_stacked

def calculate_volume_var(df_volume, past_start_day, past_end_day):
    df_volume.replace(0, np.nan, inplace=True)
    df_volume.replace([np.inf, -np.inf], np.nan, inplace=True)

    volume_var= df_volume.shift(past_end_day) / df_volume.shift(past_start_day)
    volume_var_stacked = hf.stack(volume_var, f'input_volume_var_{past_start_day}-{past_end_day}d')

    return volume_var_stacked

def calculate_market_var(df, past_days):
    market_var = hf.calculate_market_variations(df, past_days)
    market_var_stacked = hf.stack(market_var, f'input_market_var_{past_days}d')

    return market_var_stacked

def min_max_var(df, past_days):
    rolling_min = df.rolling(window=past_days + 1, min_periods=1).min()
    min_var = df / rolling_min
    min_var_stacked = hf.stack(min_var, f'input_min_var_past_{past_days}d')

    rolling_max = df.rolling(window=past_days + 1, min_periods=1).max()
    max_var = df / rolling_max
    max_var_stacked = hf.stack(max_var, f'input_max_var_past_{past_days}d')

    return min_var_stacked, max_var_stacked

def get_future_end_var(df_buy, df_sell, future_days):
    df_future_end = df_sell.shift(-future_days)
    future_end_var =  df_future_end / df_buy
    future_end_var_stacked = hf.stack(future_end_var, f'output_future_end_var')
    
    return future_end_var_stacked

def get_future_max_var(df_buy, df_sell, future_days):
    future_rolling_max = hf.get_future_rolling_max(df_sell, future_days+1)
    future_max_var = future_rolling_max / df_buy
    future_max_var_stacked = hf.stack(future_max_var, f'output_future_max_var')
        
    return future_max_var_stacked

def days_since_min_max(df, past_days):
    days_since_min = hf.get_days_since_min(df, past_days)
    days_since_min_stacked = hf.stack(days_since_min, f'input_days_since_min_{past_days}d')

    days_since_max = hf.get_days_since_max(df, past_days)
    days_since_max_stacked = hf.stack(days_since_max, f'input_days_since_max_{past_days}d')

    return days_since_min_stacked, days_since_max_stacked

def get_volatility(df, past_days):
    volatility = hf.calculate_volatility(df, past_days)
    volatility_stacked = hf.stack(volatility, f'input_volatility_{past_days}d')

    return volatility_stacked

def get_market_volatility(df, past_days):
    market_average = hf.calculate_averages(df)
    volatility = hf.calculate_volatility(market_average, past_days)
    volatility_stacked = hf.stack(volatility, f'input_market_volatility_{past_days}d')

    return volatility_stacked

def get_volume_volability(df, past_days):
    volatility = hf.calculate_volatility(df, past_days)
    volatility_stacked = hf.stack(volatility, f'input_volume_volatility_{past_days}d')

    return volatility_stacked

def get_n_ups(df, past_days):
    n_ups = hf.calculate_n_ups(df, past_days)
    n_ups_stacked = hf.stack(n_ups, f'input_n_ups_{past_days}d')

    return n_ups_stacked

def get_rank(df, past_days):
    rank = hf.calculate_rank(df, past_days)   
    rank_stacked = hf.stack(rank, f'input_rank_{past_days}d')
    
    return rank_stacked

def get_performance_vs_market(df, past_days):
    performance_vs_market = hf.calculate_performance_vs_market(df, past_days)
    performance_vs_market_stacked = hf.stack(performance_vs_market, f'input_perf_vs_market_{past_days}d')

    return performance_vs_market_stacked

def classify_var(df_var, thresholds, col_name):
    df_thresholds = hf.classify_var(df_var, thresholds)

    df_thresholds_stacked = hf.stack(df_thresholds, col_name)
    df_thresholds_stacked = df_thresholds_stacked.droplevel(level=-1)

    return df_thresholds_stacked

def get_future_min_var_before_max(df_buy, df_sell, df_low, future_days):
    rolling_max_positions = hf.get_future_rolling_max_position(df_sell, future_days)

    df_low = df_low.reset_index(drop=True)
    rolling_min = df_low.apply(lambda col: col.index.map(
            lambda row: hf.get_future_rolling_min_value(row, df_low.columns.get_loc(col.name), df_low, rolling_max_positions)
        ))
    rolling_min.index = df_buy.index
    
    var = rolling_min / df_buy
    var_stacked = hf.stack(var, f'output_future_min_before_max')

    return var_stacked


In [ ]:
def get_df_data(df_buy, df_sell, dfs_ohlcv, target_future_days):
    var_30 = calculate_var(df_buy, past_days=30, future_days=0)
    var_10 = calculate_var(df_buy, past_days=10, future_days=0)
    var_5 = calculate_var(df_buy, past_days=5, future_days=0)
    var_2 = calculate_var(df_buy, past_days=2, future_days=0)
    var_1 = calculate_var(df_buy, past_days=1, future_days=0)

    var_vs_close_1 = calculate_var_vs_past_ohlcv(df_buy, dfs_ohlcv['df_close'], past_days=1, title='close')
    var_vs_low_1 = calculate_var_vs_past_ohlcv(df_buy, dfs_ohlcv['df_low'], past_days=1, title='low')
    var_vs_high_1 = calculate_var_vs_past_ohlcv(df_buy, dfs_ohlcv['df_high'], past_days=1, title='high')

    volume_var_2_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=2, past_end_day=1)
    volume_var_3_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=3, past_end_day=1)

    # market_var_30 = calculate_market_var(df_buy, past_days=30)
    # market_var_5 = calculate_market_var(df_buy, past_days=5)
    # market_var_1 = calculate_market_var(df_buy, past_days=1)
    
    min_var_30, max_var_30 = min_max_var(df_buy, past_days=30)
    min_var_10, max_var_10 = min_max_var(df_buy, past_days=10)
    min_var_5, max_var_5 = min_max_var(df_buy, past_days=5)
    min_var_2, max_var_2 = min_max_var(df_buy, past_days=2)

    days_since_min_30, days_since_max_30 = days_since_min_max(df_buy, past_days=30)
    days_since_min_10, days_since_max_10 = days_since_min_max(df_buy, past_days=10)

    # volatility_30 = get_volatility(df_buy, past_days=30)
    # volatility_10 = get_volatility(df_buy, past_days=10)
    # volatility_2 = get_volatility(df_buy, past_days=2)

    # market_volatility_30 = get_market_volatility(df_buy, past_days=30)
    # market_volatility_10 = get_market_volatility(df_buy, past_days=10)
    # market_volatility_2 = get_market_volatility(df_buy, past_days=2)

    volume_volability_30 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=30)
    volume_volability_10 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=10)
    volume_volability_2 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=2)

    n_ups_30 = get_n_ups(df_buy, past_days=30)
    n_ups_5 = get_n_ups(df_buy, past_days=5)

    rank_30 = get_rank(df_buy, past_days=30)
    rank_10 = get_rank(df_buy, past_days=10)
    rank_5 = get_rank(df_buy, past_days=5)
    rank_2 = get_rank(df_buy, past_days=2)
    rank_1 = get_rank(df_buy, past_days=1)

    perf_vs_market_30 = get_performance_vs_market(df_buy, past_days=30)
    perf_vs_market_10 = get_performance_vs_market(df_buy, past_days=10)
    perf_vs_market_5 = get_performance_vs_market(df_buy, past_days=5)
    perf_vs_market_2 = get_performance_vs_market(df_buy, past_days=2)
    perf_vs_market_1 = get_performance_vs_market(df_buy, past_days=1)

    future_end_var = get_future_end_var(df_buy, df_sell, target_future_days)
    future_max_var = get_future_max_var(df_buy, df_sell, target_future_days)
    output_class = classify_var(future_max_var, thresholds, 'output_class')
    future_min_var = get_future_min_var_before_max(df_buy, df_sell, dfs_ohlcv['df_low'], target_future_days)

    df_data = pd.concat(
            [var_30, var_10, var_5, var_2, var_1,
            var_vs_close_1, var_vs_high_1, var_vs_low_1,
            volume_var_2_1, volume_var_3_1,
            # market_var_30, market_var_5, market_var_1,
            min_var_30, min_var_10, min_var_5, min_var_2,
            max_var_30, max_var_10, max_var_5, max_var_2,
            days_since_min_30, days_since_min_10,
            days_since_max_30, days_since_max_10,
            # volatility_30, volatility_10, volatility_2,
            # market_volatility_30, market_volatility_10, market_volatility_2,
            volume_volability_30, volume_volability_10, volume_volability_2,
            n_ups_30, n_ups_5,
            rank_30, rank_10, rank_5, rank_2, rank_1,
            perf_vs_market_30, perf_vs_market_10, perf_vs_market_5,
            perf_vs_market_2, perf_vs_market_1,
            future_end_var, future_max_var, output_class, future_min_var],
        axis='columns')

    df_data = df_data.dropna()

    return df_data

def get_dfs_input_output(df_data):
    input_columns = [col for col in df_data.columns if col.startswith('input_')]
    df_input = df_data[input_columns]
    df_output = df_data[['output_class']]

    return df_input, df_output

In [ ]:
def get_class_cumulative_percentages(y_test):
    unique_values, counts = np.unique(y_test, return_counts=True)
    percentages = counts / len(y_test)
    percentages = percentages[np.argsort(unique_values)]
    cumulative_percentages = np.cumsum(percentages)

    print(f'Market cumulative percentages per class: {cumulative_percentages}')

    return cumulative_percentages

In [ ]:
def get_test_train_data(df_input, df_output):
    # X_all = StandardScaler().fit_transform(df_input)
    # y_all = df_output.values.ravel()

    # X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.8, test_size=0.2, random_state=42)
    # y_train = y_train.astype(int)
    # y_test = y_test.astype(int)

    X_test = df_input[-test_size:].values
    y_test = df_output[-test_size:].values.ravel().astype(int)

    X_train = df_input[:-test_size].values
    y_train = df_output[:-test_size].values.ravel().astype(int)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def create_model(X_train, X_test, y_train, y_test,
                size_layer_1, size_layer_2, size_layer_3,
                dropout_rate, balance_data, batch_size):
    last_layers_size = len(thresholds) + 1

    model = Sequential()

    model.add(Dense(size_layer_1, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(size_layer_2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(size_layer_3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(last_layers_size, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    if (balance_data):
        counter = Counter(y_train)
        max_count = max(counter.values())
        class_weights = {cls: max_count / count for cls, count in counter.items()}
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), class_weight=class_weights)
    else:
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

    return model

In [ ]:
def get_df_test_vs_prediction(model, X_test, y_test,
                          predicted_n_first_classes, accepted_n_first_classes,
                          cumulated_probs_target):
    y_prediction = model.predict(X_test)
    df_prediction = pd.DataFrame(y_prediction, columns=['prediction_prob_0', 'prediction_prob_1', 'prediction_prob_2', 'prediction_prob_3', 'prediction_prob_4'])
    df_test = pd.DataFrame({'output_class': y_test})

    df_comparison = pd.concat([df_prediction, df_test], axis=1)
    df_comparison['prediction_cumulated_probs'] = df_comparison.iloc[:, :predicted_n_first_classes + 1].sum(axis=1)
    df_comparison['prediction_bool'] = (df_comparison['prediction_cumulated_probs'] > cumulated_probs_target)
    df_comparison['output_bool'] = (df_comparison['output_class'] <= accepted_n_first_classes)
    
    return df_comparison

def get_model_performance(df_comparison):
    tp = ((df_comparison['output_bool'] == True) & (df_comparison['prediction_bool'] == True)).sum()
    tn = ((df_comparison['output_bool'] == False) & (df_comparison['prediction_bool'] == False)).sum()
    fp = ((df_comparison['output_bool'] == False) & (df_comparison['prediction_bool'] == True)).sum()
    fn = ((df_comparison['output_bool'] == True) & (df_comparison['prediction_bool'] == False)).sum()

    return tp, tn, fp, fn

In [ ]:
def get_df_prediction_true(df_data, df_test_vs_prediction):
    df_data = df_data[-test_size:]
    df_test_vs_prediction.set_index(df_data.index, inplace=True)

    df = pd.concat([df_data, df_test_vs_prediction], axis='columns')
    df = df[df['prediction_bool'] == True]
    df = df[['output_future_end_var', 'output_future_min_before_max', 'output_bool']]
    
    return df

def simulate_loss_limits(df, accepted_n_first_classes):
    fee_coef = hf.get_fee_coef(fee)
    accepted_var = thresholds[accepted_n_first_classes]

    results = []

    for loss_limit in loss_limits:
        capped_profits, uncapped_profits = [], []
        win_count, loss_count, neutral_count = 0, 0, 0

        for index, row in df.iterrows():
            if row['output_future_min_before_max'] <= loss_limit:
                capped_profit = loss_limit
                uncapped_profit = loss_limit
                loss_count += 1
            elif (row['output_bool'] == True):
                capped_profit = accepted_var
                uncapped_profit = row['output_future_end_var']
                win_count += 1
            else:
                capped_profit = row['output_future_end_var']
                uncapped_profit = row['output_future_end_var']
                neutral_count += 1
            
            capped_profit *= fee_coef
            uncapped_profit *= fee_coef

            capped_profits.append(capped_profit)
            uncapped_profits.append(uncapped_profit)

        results.append({
            'loss_limit': loss_limit,
            'win_count': win_count, 'loss_count': loss_count, 'neutral_count': neutral_count,
            'capped_profit_product': np.prod(capped_profits),
            'capped_profit_average': np.mean(capped_profits),
            'capped_profit_median': np.median(capped_profits),
            'uncapped_profit_product': np.prod(uncapped_profits),
            'uncapped_profit_average': np.mean(uncapped_profits),
            'uncapped_profit_median': np.median(uncapped_profits)
        })

    df_results = pd.DataFrame(results)
    print(df_results.head(100))

In [ ]:
from itertools import product

results = []

for params in product(*param_grid.values()):
    hyperparams = dict(zip(param_grid.keys(), params))
    buying_time = hyperparams['buying_times']
    selling_time = hyperparams['selling_times']
    target_future_days = hyperparams['target_future_days']
    size_layer_1 = hyperparams['sizes_layer_1']
    size_layer_2 = hyperparams['sizes_layer_2']
    size_layer_3 = hyperparams['sizes_layer_3']
    dropout_rate = hyperparams['dropout_rates']
    balance_data = hyperparams['balance_data']
    batch_size = hyperparams['batch_sizes']
    predicted_n_first_classes = hyperparams['n_first_classes'][0]
    accepted_n_first_classes = hyperparams['n_first_classes'][1]
    cumulated_probs_target = hyperparams['cumulated_probs_targets']

    df_buy = get_single_level_df(df, buying_time)
    df_sell = get_single_level_df(df, selling_time)
    dfs_ohlcv = get_ohlcv_dfs(df)

    if os.path.exists(transformed_data_path):
        df_data = pd.read_pickle(transformed_data_path)
        print(f'using existing {transformed_data_path}')
    else:
        print(f'need to create {transformed_data_path}')
        df_data = get_df_data(df_buy, df_sell, dfs_ohlcv, target_future_days)
        df_data.to_pickle(transformed_data_path)

    df_input, df_output = get_dfs_input_output(df_data)

    X_train, X_test, y_train, y_test = get_test_train_data(df_input, df_output)

    class_cumulative_percentages = get_class_cumulative_percentages(y_test)
    market_rate = class_cumulative_percentages[accepted_n_first_classes]

    trained_model = create_model(X_train, X_test, y_train, y_test,
                                size_layer_1, size_layer_2, size_layer_3,
                                dropout_rate, balance_data, batch_size)
    
    df_test_vs_prediction = get_df_test_vs_prediction(trained_model, X_test, y_test,
                                           predicted_n_first_classes, accepted_n_first_classes,
                                           cumulated_probs_target)
    tp, tn, fp, fn = get_model_performance(df_test_vs_prediction)
    
    winning_rate = float(tp / (tp + fp))
    winning_rate_vs_market = winning_rate - market_rate
    
    result = {
        'winning_rate': winning_rate,
        'market_rate': market_rate, 'winning_rate_vs_market': winning_rate_vs_market,
        'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn,
        'cumulated_probs_target': cumulated_probs_target,
        'predicted_n_first_classes': predicted_n_first_classes,
        'accepted_n_first_classes': accepted_n_first_classes,
        'target_future_days': target_future_days,
        'buying_time': buying_time, 'selling_time': selling_time,
        'size_layer_1': size_layer_1, 'size_layer_2': size_layer_2, 'size_layer_3': size_layer_3,
        'dropout_rate': dropout_rate, 'balance_data': balance_data, 'batch_size': batch_size,
        'thresholds': thresholds, 'epochs': epochs
    }
    print(result)
    results.append(result)
 
    df_prediction_true = get_df_prediction_true(df_data, df_test_vs_prediction)
    simulate_loss_limits(df_prediction_true, accepted_n_first_classes)

df_results = pd.DataFrame(results)
df_results.sort_values(by='winning_rate', ascending=False)
df_results.head(100)

In [ ]:
# search_space = {
#     'buying_time': hp.choice('buying_time', ['Open', 'Close']),
#     'selling_time': hp.choice('selling_time', ['Open', 'Close', 'High']),
#     'target_future_days': hp.quniform('target_future_days', 1, 20, 1),
#     'size_layer_1': hp.choice('size_layer_1', [64, 128, 256]),
#     'size_layer_2': hp.choice('size_layer_2', [64, 128, 256]),
#     'size_layer_3': hp.choice('size_layer_3', [64, 128, 256]),
#     'dropout_rate': hp.uniform('dropout_rate', 0, 0.3),
#     'balance_data': hp.choice('balance_data', [True, False]),
#     'batch_size': hp.choice('batch_size', [32, 64, 128]),
#     'n_first_classes': hp.choice('n_first_classes', [[0, 0], [0, 1], [0, 2], [1, 1], [1, 2], [2, 2]]),
#     'cumulated_probs_target': hp.uniform('cumulated_probs_target', 0.4, 1),
# }

# search_space = {
#     'buying_time': hp.choice('buying_time', ['Open']),
#     'selling_time': hp.choice('selling_time', ['High']),
#     'target_future_days': hp.quniform('target_future_days', 1, 25, 1),
#     'size_layer_1': hp.choice('size_layer_1', [64, 128, 256]),
#     'size_layer_2': hp.choice('size_layer_2', [64, 128, 256]),
#     'size_layer_3': hp.choice('size_layer_3', [64, 128, 256]),
#     'dropout_rate': hp.uniform('dropout_rate', 0, 0.3),
#     'balance_data': hp.choice('balance_data', [True, False]),
#     'batch_size': hp.choice('batch_size', [32, 64, 128]),
#     'n_first_classes': hp.choice('n_first_classes', [[0, 0], [0, 1], [0, 2], [1, 1], [1, 2], [2, 2]]),
#     'cumulated_probs_target': hp.uniform('cumulated_probs_target', 0.4, 1),
# }

# results = []

# def objective(params):
#     buying_time = params['buying_time']
#     selling_time = params['selling_time']
#     target_future_days = int(params['target_future_days'])
#     size_layer_1 = params['size_layer_1']
#     size_layer_2 = params['size_layer_2']
#     size_layer_3 = params['size_layer_3']
#     dropout_rate = params['dropout_rate']
#     balance_data = params['balance_data']
#     batch_size = params['batch_size']
#     predicted_n_first_classes = params['n_first_classes'][0]
#     accepted_n_first_classes = params['n_first_classes'][1]
#     cumulated_probs_target = params['cumulated_probs_target']

#     df_buy, df_sell = get_dfs_buy_sell(buying_time, selling_time)
#     df_input, df_output = get_dfs_input_output(df_buy, df_sell, target_future_days)
#     # class_cumulative_percentages = get_class_cumulative_percentages(df_output)

#     X_train, X_test, y_train, y_test = get_test_train_data(df_input, df_output)
#     trained_model = create_model(X_train, X_test, y_train, y_test,
#                                       size_layer_1, size_layer_2, size_layer_3,
#                                       dropout_rate, balance_data, batch_size)
#     tp, tn, fp, fn = get_model_performance(trained_model, X_test, y_test,
#                                            predicted_n_first_classes, accepted_n_first_classes,
#                                            cumulated_probs_target)

#     winning_rate = tp / (tp + fp)
    
#     tp_threshold = 400
#     performance = winning_rate if tp > tp_threshold else winning_rate * (tp / tp_threshold)

#     result = {'params': params, 'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn,
#               'winning_rate': winning_rate, 'performance': performance}
#     results.append(result)

#     return -performance

# n_iter = 20

# # rstate = hyperopt.RandomState(seed=42, print_node=lambda s: print(s, end="", flush=True))
# best = fmin(objective, search_space, algo=tpe.suggest, max_evals=n_iter)

# print('All results:')
# df_results = pd.DataFrame(results)
# results.head(1000)

# print("Best parameters:")
# print(best)